In [1]:
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import rioxarray as rio
import xarray as xr
from rasterstats import zonal_stats

plt.style.use("ggplot")

# Updating critical loads for water

## Part D: Summaries per BLR

This notebook summarises the updated water chemistry for each BLR grid cell and also calculates some other spatial statistics needed for the critical loads calculations.

#### Note

I have some doubts regarding the quality of the GAMs generated in Part B. In particular, although the model diagnostics are not too bad, it looks to me as though the models might be overfitting the data - especially the one for nitrate. It is also possible that some of the data used for temporal trends is misleading: it seems that the "tidstrendsjøer" are not easily available via Aquamonitor and the 78 sites I am currently using are not exactly the same as the 78 in RESA. **I will leave this for now, but we might need to revisit these issues later and re-run**.

In [2]:
cl_eng = nivapy.da.connect_postgis(database="critical_loads")
eng = nivapy.da.connect_postgis()

Connection successful.
Connection successful.


In [3]:
# Get BLRs
blr_gdf = nivapy.da.read_postgis("deposition", "dep_grid_blr", cl_eng).to_crs(
    "epsg:25833"
)
blr_gdf = blr_gdf[["cell_id", "geom"]]
blr_gdf.head()

,cell_id,geom
0,58005004,"MULTIPOLYGON (((-25471.982 6514836.998, -25514..."
1,58006001,"MULTIPOLYGON (((-10886.678 6503779.108, -20006..."
2,58006002,"MULTIPOLYGON (((3660.479 6501900.217, -10886.6..."
3,58006003,"MULTIPOLYGON (((18211.985 6500076.038, 3660.47..."
4,58006004,"MULTIPOLYGON (((32767.708 6498306.548, 18211.9..."


In [4]:
# Get original TOC and NO3 values from 1995
par_df = pd.read_sql(
    "SELECT id as parameter_id, name, class FROM water.parameter_definitions",
    cl_eng,
)
req_df = pd.read_sql("SELECT * FROM water.blr_required_parameters", cl_eng)
req_df = pd.merge(req_df, par_df, how="left", on="parameter_id")
del req_df["parameter_id"], req_df["class"]
req_df = req_df.pivot(index="region_id", columns="name", values="value")
req_df.index.name = "Region_id"
req_df.reset_index(inplace=True)
req_df.columns.name = ""
req_df = req_df[["Region_id", "NO3N", "TOC"]]
req_df.columns = ["cell_id", "NO3_NO2_ugpl_orig", "TOC_mgpl_orig"]

blr_gdf = blr_gdf.merge(req_df, how="left", on="cell_id")
blr_gdf.head()

,cell_id,geom,NO3_NO2_ugpl_orig,TOC_mgpl_orig
0,58005004,"MULTIPOLYGON (((-25471.982 6514836.998, -25514...",NaN,NaN
1,58006001,"MULTIPOLYGON (((-10886.678 6503779.108, -20006...",365.0,0.4
2,58006002,"MULTIPOLYGON (((3660.479 6501900.217, -10886.6...",215.0,0.8
3,58006003,"MULTIPOLYGON (((18211.985 6500076.038, 3660.47...",200.0,0.6
4,58006004,"MULTIPOLYGON (((32767.708 6498306.548, 18211.9...",300.0,0.8


## 1. TOC and NO3

Using the statistically interpolated 1 km grids for TOC and NO3, the code below calculates average concentrations per BLR cell per year. For TOC we have only interpolated data for 2019, whereas for NO3 we have annual estimates for the period from 1990 to 2019.

In [6]:
st_yr = 1990
end_yr = 2019

years = range(st_yr, end_yr + 1)
stats = ["mean"]

# Zonal stats.
path_dict = {
    "TOC_mgpl": r"/home/jovyan/shared/critical_loads/norway_interpolated_toc_2019.nc",
    "NO3_NO2_ugpl": r"/home/jovyan/shared/critical_loads/norway_interpolated_no3_1990-2021.nc",
}

for year in years:
    for par, nc_path in path_dict.items():
        if (par == "TOC_mgpl") and (year != 2019):
            # Only have data for 2019
            pass
        else:
            with xr.open_dataset(nc_path, mask_and_scale=True) as ds:
                # rasterstats can't handle arrays indexed with increasing 'y'. See
                # https://github.com/perrygeo/python-rasterstats/issues/98
                # Flip array and 'y' indices to make compatible
                ds = ds.sel(y=slice(None, None, -1))
                ds = ds.rio.write_transform()
                ds[par].sel(year=year).rio.to_raster("temp.tif")

            stats_df = pd.DataFrame(
                zonal_stats(
                    vectors=blr_gdf,
                    raster="temp.tif",
                    stats=stats,
                    all_touched=False,
                )
            )
            blr_gdf[f"{par}_{year}"] = stats_df["mean"]

os.remove("temp.tif")
blr_gdf.head()

KeyboardInterrupt: 

The code below plots the new TOC grid for 2019 against the old one. It also plots the new NO3 grid for the specified year against the original.

In [ ]:
# Plot
no3_year = 1990

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
blr_gdf.plot(column="TOC_mgpl_orig", legend=True, ax=axes[0, 0], vmax=25)
blr_gdf.plot(column="TOC_mgpl_2019", legend=True, ax=axes[0, 1], vmax=25)
blr_gdf.plot(column="NO3_NO2_ugpl_orig", legend=True, ax=axes[1, 0], vmax=200)
blr_gdf.plot(column=f"NO3_NO2_ugpl_{no3_year}", legend=True, ax=axes[1, 1], vmax=200)
axes[0, 0].set_title("TOC by BLR (original; mg/l)")
axes[0, 1].set_title("TOC by BLR in 2019 (mg/l)")
axes[1, 0].set_title("NO3 by BLR in (original; ugN/l)")
axes[1, 1].set_title(f"NO3 by BLR in {no3_year} (ugN/l)")
for ax in axes.flatten():
    ax.axis("off")
plt.tight_layout()

Overall the results above seem reasonable: TOC concentrations have increased more-or-less as expected, and the NO3 grid for 1990 from the new approach seems broadly compatible with the old version. The new approach also produces smoother, more complete results than the old version, and there are fewer large outliers.

One feature of the new NO3 grid that seems questionable is the "island" of high concentrations in south-central Norway, a little north of Dombås (near Dovrefjell National Park). This seems to be caused by high nitrate concentrations measured at a single site - Kjelsungvatnet. However, after some investigation we can find no reason to exclude these data, so we will assume the signal is real. See e-mail from Kari received 09.02.2023 at 12.41 for details. 

## 2. Runoff

The code in this section estimates mean annual runoff in mm/yr based on NVE's runoff "normal" from 1991 to 2020.

In [ ]:
ro_grid_path = r"/home/jovyan/shared/teotil3/nve_hbv_data/runoff_1991-2020_mm_year.tif"
stats = ["mean"]
stats_df = pd.DataFrame(
    zonal_stats(
        vectors=blr_gdf,
        raster=ro_grid_path,
        stats=stats,
        all_touched=False,
    )
)
blr_gdf["runoff_mmpyr"] = stats_df["mean"]

# Plot
ax = blr_gdf.plot(column="runoff_mmpyr", legend=True, figsize=(8, 8))
ax.axis("off");

## 3. AR50 proportions

We need to calculate the following for each BLR cell:

 * Cell area
 * Lake area (AR50 class 81)
 * Forest area (AR50 class 30
 * Bare area (AR50 classes 51 and 70)
 * Peat area (AR50 class 60)
 
Note that all of these are simple/high-level divisions of `artype`, except class 51, which is a subdivision within `arveget`. The code below calculates a new column where class 51 is used where present, otherwise `artype`.

In [ ]:
print("Reprojecting...")
clip_gdf = blr_gdf[["cell_id", "geom"]].to_crs({"proj": "cea"}).copy()
ar50_gdf = nivapy.da.read_postgis("physical", "norway_nibio_ar50_poly", eng).to_crs(
    {"proj": "cea"}
)

print("Reclassifying...")
class_dict = {
    "81": "Lake_area",
    "30": "Forest_area",
    "51": "Bare_area",
    "70": "Bare_area",
    "60": "Peat_area",
    "-1": "Other_area",
}
ar50_gdf["lc_class"] = np.where(
    ar50_gdf["arveget"] == 51, ar50_gdf["arveget"], ar50_gdf["artype"]
)
ar50_gdf["lc_class"] = np.where(
    ar50_gdf["lc_class"].isin(class_dict.keys()), ar50_gdf["lc_class"], "-1"
)
ar50_gdf["lc_class"].replace(class_dict, inplace=True)

print("Intersecting...")
int_gdf = gpd.overlay(clip_gdf, ar50_gdf, how="intersection", keep_geom_type=True)
int_gdf["area_km2"] = int_gdf["geometry"].area / 1e6

print("Aggregating...")
lc_df = int_gdf.groupby(["cell_id", "lc_class"]).sum()["area_km2"]
lc_df = lc_df.unstack("lc_class")
lc_df.columns = [f"{i}_km2" for i in lc_df.columns]
lc_df.fillna(0, inplace=True)
lc_df["Total_area_km2"] = lc_df.sum(axis="columns")
lc_df.reset_index(inplace=True)
lc_df.columns.name = ""

blr_gdf = blr_gdf.merge(lc_df, how="left", on="cell_id")
blr_gdf.dropna(how="any", inplace=True)
blr_gdf.head()

In [ ]:
# Save
blr_csv = "updated_blr_summary.csv"
blr_df = pd.DataFrame(blr_gdf.drop(columns="geom"))
blr_df.to_csv(blr_csv, index=False)